# Biblioteca

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import arrow
import datetime
import pytz
%matplotlib inline

# Procurando os dados entre os meses de março e abril

In [ ]:
ts = []
ts1 = []
dia       = []
cont_MES  = []
cont0     = []
diretorio = ["/data2/raw/*","/data3/raw/*"]
ref1      = ['/data2/raw/AngraRun_T','/data3/raw/AngraRun_T']

for j in range(len(diretorio)):
    List = glob.glob(diretorio[j]) # listando todos os dados
    for i in range(len(List)):
        if List[i][0:21] != ref1[j]:
            cont0.append(List[i])
for i in range(len(cont0)):
    ts.append(int(np.array(cont0)[i][20:-8]))
    ts1.append(np.array(cont0)[i])

In [ ]:
ts=np.sort(ts)
ind=np.argsort(ts)

In [2]:
def contron_pac(mes):


    dia       = []
    cont_MES  = []
    cont0     = []
    diretorio = ["/data2/raw/*","/data3/raw/*"]
    ref1      = ['/data2/raw/AngraRun_T','/data3/raw/AngraRun_T']

    for j in range(len(diretorio)):
        List = glob.glob(diretorio[j]) # listando todos os dados
        for i in range(len(List)):
            if List[i][0:21] != ref1[j]:
                cont0.append(List[i])
        for i in range(len(cont0)):
            ts = int(np.array(cont0)[i][20:-8])
            datas  = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
            if datas[0:4] == '2019':
                if datas[5:7] == mes:
                    cont_MES.append(cont0[i])
                    if datas[8] == '0':
                        dia.append(datas[9])
                    if datas[8] == '1':
                        dia.append(datas[8:10])
                    if datas[8] == '2':
                        dia.append(datas[8:10])
                    if datas[8] == '3':
                        dia.append(datas[8:10])
    
    #cont0     = []
  ##  mes_aux =['04','06','07']                  
   # if np.sum(np.array(mes_aux) == mes) >0:
   #     List  = glob.glob('/data1/data3/raw/*') # listando todos os dados
  #      for i in range(len(List)):
  #          ts     = int(np.array(List)[i][34:-8])
  #          datas  = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')
   #         if datas[0:4] == '2019':
  #              if datas[5:7] == mes:
    #                cont_MES.append(List[i])
   #                 if datas[8] == '0':
   #                     dia.append(datas[9])
   #                 if datas[8] == '1':
    #                    dia.append(datas[8:10])
    #                if datas[8] == '2':
    #                    dia.append(datas[8:10])
      ##              if datas[8] == '3':
                     #   dia.append(datas[8:10])

    #cont_MES.append(List)
                    
    return cont_MES,dia

In [3]:
def dist_pedestais_Corrigida(df,STS,Num_NDAQ,flag):
    if flag == 1:
        ind_NDAQ                     = np.where(df['Slot Number'] == Num_NDAQ)
        S                            = np.stack(df[STS])[ind_NDAQ]
    else:
        S = df
    # parte 1
    min_sinal                    = np.min(S,axis = 1).reshape(len(S),1)                                                       # valores minimos dos sinais
    r                            = np.zeros((1,len(min_sinal)))[0]                                                            # Vetor dos pedestais dos sinais
    ind_part1                    = np.where(np.sum(S[:,0:5] - min_sinal > 4,axis = 1) == 0)[0]                                # nn passo de < 4 adc ate os 5 primeiros pontos dos sinais
    r[ind_part1]                 = np.mean(S[ind_part1,0:5],axis = 1)                                                         # valores dos pedestais dos sinais da primeira etapa
    ind_part1_cont               = np.where(np.sum(S[:,0:5] - min_sinal > 4,axis = 1) > 0)[0]                                 # passo dos 4 adc nos Sinal[0:5]
    # parte 2
    ind_part2                    = np.where(np.sum(S[ind_part1_cont,45:50] - min_sinal[ind_part1_cont] > 4,axis = 1) == 0)[0] # nn passo dos 4 adc Sinal[45:50]
    r[ind_part1_cont[ind_part2]] = np.mean(S[ind_part1_cont[ind_part2],45:50],axis = 1)
    # parte 3
    g                            = np.zeros((1,len(min_sinal)))[0]                                                            # Sinais que não atigim a condição da etapa 1 e da etapa 2
    g[ind_part1]                 = 1
    g[ind_part1_cont[ind_part2]] = 1
    ind_part3                    = np.where(np.ones((1,len(min_sinal)))[0] - g ==1)[0]                                        # Sinais que nn atigiram a condição da parte 1 e 2
    for i in range(0,len(ind_part3)):
        par = np.sum(S[ind_part3][i].reshape(10,5) - min_sinal[ind_part3][i] >4,axis = 1) 
        if np.sum(par == 0) > 0:
            k         = np.where(par == 0)[0]
            r[ind_part3[i]] = np.mean(S[ind_part3][i,5*(k[0]):5*(k[0]) + 5])
        else:
            k               = np.argmin(par)
            r[ind_part3[i]] = min_sinal[ind_part3][i] + 2 
    
    return r


def carga(df,STS,canal):
    df         = np.stack(df[STS])
    pos_amost  = np.argmax(df[canal], axis = 1)
    ind        = np.where((pos_amost > 9) & (pos_amost<20))
    df         = df[ind]
    Pedestal   = dist_pedestais_Corrigida(df,STS,Num_NDAQ,flag=0)
    Amp        = np.max(df-Pedestal.reshape((len(Pedestal),1)), axis =1)
    Amp        = np.max(df, axis = 1)
    df         = df[Amp>3]
    cg         = np.sum(df, axis = 1)
    return cg

# Olhando os dias que não houve nenhuma opereção em teste run

In [4]:
MES = ['02']

In [5]:
D = []

In [8]:
for i in range(len(MES)):
    aux          = []
    mes          = MES[i]
    cont_MES,dia = contron_pac(mes)
    d = np.unique(np.int64(dia))
    #for j in range(len(d)):
    #    pac  = np.array(list(cont_MES))[np.int64(dia) == d[j]]
    #    if len(pac) > 40:
    #        aux.append(d[j])
    #D.append(np.array(aux))

In [9]:
d

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28])

# Criando um dataflame com as informações de operação do detector

In [ ]:
mar = ['01-03','02-03','03-03','04-03','05-03','06-03','07-03','08-03','09-03','10-03',
         '11-03','12-03','13-03','14-03','15-03','16-03','17-03','18-03','19-03','20-03',
         '21-03','22-03','23-03','24-03','25-03','26-03','27-03','28-03','29-03','30-03','31-03']

abril = ['01-04','02-04','03-04','04-04','05-04','06-04','07-04','08-04','09-04','10-04',
         '11-04','12-04','13-04','14-04','15-04','16-04','17-04','18-04','19-04','20-04',
         '21-04','22-04','23-04','24-04','25-04','26-04','27-04','28-04','29-04','30-04']

maio = ['01-05','02-05','03-05','04-05','05-05','06-05','07-05','08-05','09-05','10-05',
        '11-05','12-05','13-05','14-05','15-05','16-05','17-05','18-05','19-05','20-05',
        '21-05','22-05','23-05','24-05','25-05','26-05','27-05','28-05','29-05','30-05','31-05']

Jun = ['01-06','02-06','03-06','04-06','05-06','06-06','07-06','08-06','09-06','10-06',
       '11-06','12-06','13-06','14-06','15-06','16-06','17-06','18-06','19-06','20-06',
       '21-06','22-06','23-06','24-06','25-06','26-06','27-06','28-06','29-06','30-06']

Jul = ['01-07','02-07','03-07','04-07','05-07','06-07','07-07','08-07','09-07','10-07',
       '11-07','12-07','13-07','14-07','15-07','16-07','17-07','18-07','19-07','20-07',
       '21-07','22-07','23-07','24-07','25-07','26-07','27-07','28-07','29-07','30-07','31-07']

Agos = ['01-08','02-08','03-08','04-08','05-08','06-08','07-08','08-08','09-08','10-08',
        '11-08','12-08','13-08','14-08','15-08','16-08','17-08','18-08','19-08','20-08',
        '21-08','22-08','23-08','24-08','25-08','26-08','27-08','28-08','29-08','30-08','31-08']

Set = ['01-09','02-09','03-09','04-09','05-09','06-09','07-09','08-09','09-09','10-09',
       '11-09','12-09','13-09','14-09','15-09','16-09','17-09','18-09','19-09','20-09',
       '21-09','22-09','23-09','24-09','25-09','26-09','27-09','28-09','29-09','30-09']


In [ ]:
#mes_março    = data_flame (D[0],mar)  # junho

In [ ]:
#mar

abril

In [ ]:
mes_abril = 'Teste'
mes_abril = np.repeat(np.array(mes_abril),len(abril))
mes_abril[np.array(D[0])-1] = 'On'
mes_abril[21:] = 'Off'

maio

In [ ]:
mes_maio = 'Teste'
mes_maio = np.repeat(np.array(mes_maio),len(maio))
mes_maio[np.array(D[1])-1] = 'On'
mes_maio[:23] = 'Off'

Outros meses

In [ ]:
def data_flame (DD,aux_mes):
    mes = 'Teste'
    mes = np.repeat(np.array(mes),len(aux_mes))
    mes[np.array(DD)-1] = 'On'
   
    return mes

In [ ]:
mes_junho    = data_flame (D[2],Jun)  # junho
mes_julho    = data_flame (D[3],Jul)  # julho
mes_agosto   = data_flame (D[4],Agos) # Agosto
mes_setembro = data_flame (D[5],Set) # Setembro

In [ ]:
ope = np.concatenate((mes_abril,mes_maio,mes_junho,mes_julho,mes_agosto,mes_setembro))
datas = np.concatenate((abril,maio,Jun,Jul,Agos,Set))

In [ ]:
Testes = np.concatenate((datas.reshape(-1,1),ope.reshape(-1,1)), axis = 1)

In [23]:
df = pd.DataFrame(Testes,columns=['Data','Operação'])

In [29]:
df.to_parquet('/home/antoniojr/notebooks/monitoramento temporal/info')

In [28]:
np.unique(df.Operação.values)

array(['Off', 'On', 'Teste'], dtype=object)